In [44]:
#--------------------QRE background--------------------#

print('Running QRE')
import numpy as np
import pandas as pd

# Parameters
vh = 200
vl = 100
ph = 160
pl =  80
ch =  40
cl =   0

m1 = (ph - vl) / (vh - vl)
m2 = (ph - pl) / (vh - vl)
kl  = (pl - cl) / (ph - cl)
kh  = (pl - ch) / (ph - ch)
x1  = 1 / (2 * m1)
x2  = 1 / (2 * m2)
x3  = 1 - x2

#--------------------n = 1 QRE--------------------#
print('Running QRE for n = 1...')

# Structure of y
# y[0] = pbhh
# y[1] = pbdh
# y[2] = pbll
# y[3] = pbdl
# y[4] = pshh
# y[5] = pslh
# y[6] = pshl
# y[7] = psll
# y[8] = l

# Expected utility
def muh(y):
    return y[4] / (y[4] + y[6])
def mul(y):
    return y[5] / (y[5] + y[7])
def ubh(y):
    return vl - ph + (vh - vl) * muh(y)
def ubl(y):
    return vl - pl + (vh - vl) * mul(y)
def ubd(y):
    return 0
def ush(y):
    return (ph - ch) * y[0] - (pl - ch) * y[2]
def usl(y):
    return (ph - cl) * y[0] - (pl - cl) * y[2]

def H(y): # H = 0 defines a QRE
    H1 = np.log(y[0]) - np.log(y[1]) - y[8] * ubh(y)
    H3 = np.log(y[2]) - np.log(y[3]) - y[8] * ubl(y)
    H5 = np.log(y[4]) - np.log(y[5]) - y[8] * ush(y)
    H7 = np.log(y[6]) - np.log(y[7]) - y[8] * usl(y)
    H2 = y[0] + y[1] - 1
    H4 = y[2] + y[3] - 1
    H6 = y[4] + y[5] - 1
    H8 = y[6] + y[7] - 1
    return np.array([H1,H2,H3,H4,H5,H6,H7,H8])

def Jacobian(y): # dH/dy
    J1 = [      1 / y[0], -1 / y[1], 0, 0, -(vh - vl) * y[8] * y[6] / (y[4] + y[6])**2, 0, (vh - vl) * y[8] * y[4] / (y[4] + y[6])**2, 0, -ubh(y)]
    J3 = [0, 0, 1 / y[2], -1 / y[3], 0,    -(vh - vl) * y[8] * y[7] / (y[5] + y[7])**2, 0, (vh - vl) * y[8] * y[5] / (y[5] + y[7])**2,    -ubl(y)]
    J5 = [-(ph - ch) * y[8], 0, (pl - ch) * y[8], 0,       1 / y[4], -1 / y[5], 0, 0, -ush(y)]
    J7 = [-(ph - cl) * y[8], 0, (pl - cl) * y[8], 0, 0, 0, 1 / y[6], -1 / y[7],       -usl(y)]
    J2 = [1,1,0,0,0,0,0,0,0]
    J4 = [0,0,1,1,0,0,0,0,0]
    J6 = [0,0,0,0,1,1,0,0,0]
    J8 = [0,0,0,0,0,0,1,1,0]
    return np.array([J1,J2,J3,J4,J5,J6,J7,J8])

def newton_method(H, Jacobian, y0, tol=1e-6, max_iter=100):
    y = y0    
    for _ in range(max_iter):
        Hx = H(y)
        Jx = Jacobian(y)[:,:-1]
        delta_x = np.linalg.solve(Jx, -Hx)
        y[:-1] = y[:-1] + delta_x
        if np.linalg.norm(delta_x) < tol:
            break
    return y

Main_Branch_1s = [] # This is, in fact, the only branch
y0 = np.array([1/2, 1/2, 1/2, 1/2, 1/2, 1/2, 1/2, 1/2, 0])
Main_Branch_1s.append(y0)
def main_stepfn(y): # Step size
    return 0.002
total_steps = 1500
final_l = 0.6
length = len(y0)

y = y0
for step in range(total_steps):
    determinants = np.array([np.linalg.det(np.delete(Jacobian(y), i, axis=1)) for i in range(length)])
    dy = np.power(-1, np.arange(1, length + 1)) * determinants
    dy = -dy/np.linalg.norm(dy) # dy/ds
    ds = main_stepfn(y)
    y_predicted = y + ds * dy # predict new y
    y_optimized = newton_method(H, Jacobian, y_predicted) # fine-grained optimization around prediction
    Main_Branch_1s.append(y_optimized)
    y = y_optimized
    if y[-1] > final_l:
        break
Main_Branch_1s = np.array(Main_Branch_1s)

#--------------------n = 2 QRE--------------------#
print('Running QRE for n = 2...')

# Structure of y:
# y[0]  = pbhh
# y[1]  = pbdh
# y[2]  = pbll
# y[3]  = pbdl
# y[4]  = pbhb
# y[5]  = pblb
# y[6]  = pbdb
# y[7]  = pshl
# y[8]  = psll
# y[9]  = pshh
# y[10] = pslh
# y[11] = l

n = 2 # number of firms

# Expected utility
def muh(y):
    return y[9] / (y[7] + y[9])
def mul(y):
    return y[10] / (y[8] + y[10])
def x(y):
    return 0.5 * (y[7] + y[9])
def ubhh(y):
    return vl - ph + (vh - vl) * muh(y)
def ubdh(y):
    return 0
def ubll(y):
    return vl - pl + (vh - vl) * mul(y)
def ubdl(y):
    return 0
def ubhb(y):
    return vl - ph + (vh - vl) * muh(y)
def ublb(y):
    return vl - pl + (vh - vl) * mul(y)
def ubdb(y):
    return 0
def ushl(y):
    LHS = x(y)**(n-1) / n
    RHS = (1 - x(y)**n - (1-x(y))**n) / (n * x(y))
    return (ph - cl) * (LHS * y[0] + RHS * y[4])
def usll(y):
    LHS = (1-x(y))**(n-1) / n
    RHS = (1 - x(y)**n - (1-x(y))**n) / (n * (1-x(y)))
    return (pl - cl) * (LHS * y[2] + RHS * y[5])
def ushh(y):
    LHS = x(y)**(n-1) / n
    RHS = (1 - x(y)**n - (1-x(y))**n) / (n * x(y))
    return (ph - ch) * (LHS * y[0] + RHS * y[4])
def uslh(y):
    LHS = (1-x(y))**(n-1) / n
    RHS = (1 - x(y)**n - (1-x(y))**n) / (n * (1-x(y)))
    return (pl - ch) * (LHS * y[2] + RHS * y[5])

def H(y): # H = 0 defines a QRE
    H1  = np.log(y[0]) - np.log(y[ 1]) - y[11] *  ubhh(y)
    H3  = np.log(y[2]) - np.log(y[ 3]) - y[11] *  ubll(y)
    H5  = np.log(y[4]) - np.log(y[ 6]) - y[11] *  ubhb(y)
    H6  = np.log(y[5]) - np.log(y[ 6]) - y[11] *  ublb(y)
    H8  = np.log(y[9]) - np.log(y[10]) - y[11] * (ushh(y) - uslh(y))
    H10 = np.log(y[7]) - np.log(y[ 8]) - y[11] * (ushl(y) - usll(y))
    H2  = y[0] + y[ 1]        - 1
    H4  = y[2] + y[ 3]        - 1
    H7  = y[4] + y[ 5] + y[6] - 1
    H11 = y[7] + y[ 8]        - 1
    H9  = y[9] + y[10]        - 1
    return np.array([H1, H2, H3, H4, H5, H6, H7, H8, H9, H10, H11])

def Jacobian(y): # dH/dy
    J1  = [               1 / y[0],    -1 / y[1], 0, 0, 0, 0, 0, (vh - vl) * y[11] * y[ 9] / (y[7] + y[ 9])**2, 0, -(vh - vl) * y[11] * y[7] / (y[7] + y[ 9])**2, 0, -ubhh(y)]
    J3  = [0, 0,          1 / y[2],    -1 / y[3], 0, 0, 0, 0,    (vh - vl) * y[11] * y[10] / (y[8] + y[10])**2, 0, -(vh - vl) * y[11] * y[8] / (y[8] + y[10])**2,    -ubll(y)]
    J5  = [0, 0, 0, 0,    1 / y[4], 0, -1 / y[6],                (vh - vl) * y[11] * y[ 9] / (y[7] + y[ 9])**2, 0, -(vh - vl) * y[11] * y[7] / (y[7] + y[ 9])**2, 0, -ubhh(y)]
    J6  = [0, 0, 0, 0, 0, 1 / y[5],    -1 / y[6], 0,             (vh - vl) * y[11] * y[10] / (y[8] + y[10])**2, 0, -(vh - vl) * y[11] * y[8] / (y[8] + y[10])**2,    -ubll(y)]
    J2  = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    J4  = [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    J7  = [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
    J11 = [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
    J9  = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
    dushh_dx = (ph - ch) * ( (n-1) *    x(y) **(n-2) * y[0] + ( (1 + (n-1) *    x(y) ) * (1-x(y))**(n-1) - (n-1) *    x(y) **n - 1 ) * y[4] /    x(y) **2 ) / n
    duslh_dx = (pl - ch) * (-(n-1) * (1-x(y))**(n-2) * y[2] - ( (1 + (n-1) * (1-x(y))) *    x(y) **(n-1) - (n-1) * (1-x(y))**n - 1 ) * y[5] / (1-x(y))**2 ) / n
    dushl_dx = (ph - cl) * ( (n-1) *    x(y) **(n-2) * y[0] + ( (1 + (n-1) *    x(y) ) * (1-x(y))**(n-1) - (n-1) *    x(y) **n - 1 ) * y[4] /    x(y) **2 ) / n
    dusll_dx = (pl - cl) * (-(n-1) * (1-x(y))**(n-2) * y[2] - ( (1 + (n-1) * (1-x(y))) *    x(y) **(n-1) - (n-1) * (1-x(y))**n - 1 ) * y[5] / (1-x(y))**2 ) / n
    J8  = [-(ph - ch) * y[11] * x(y)**(n-1) / n, 0, (pl - ch) * y[11] * (1-x(y))**(n-1) / n, 0, -(ph - ch) * y[11] * (1 - x(y)**n - (1-x(y))**n) / (n * x(y)), (pl - ch) * y[11] * (1 - x(y)**n - (1-x(y))**n) / (n * (1-x(y))), 0,          - 0.5 * y[11] * (dushh_dx - duslh_dx), 0,         1 / y[9] - 0.5 * y[11] * (dushh_dx - duslh_dx), -1 / y[10], uslh(y) - ushh(y)]
    J10 = [-(ph - cl) * y[11] * x(y)**(n-1) / n, 0, (pl - cl) * y[11] * (1-x(y))**(n-1) / n, 0, -(ph - cl) * y[11] * (1 - x(y)**n - (1-x(y))**n) / (n * x(y)), (pl - cl) * y[11] * (1 - x(y)**n - (1-x(y))**n) / (n * (1-x(y))), 0, 1 / y[7] - 0.5 * y[11] * (dushl_dx - dusll_dx), -1 / y[8],          - 0.5 * y[11] * (dushl_dx - dusll_dx), 0,          usll(y) - ushl(y)]
    return np.array([J1, J2, J3, J4, J5, J6, J7, J8, J9, J10, J11])

Main_Branch_2s = []
y0 = np.array([1/2, 1/2, 1/2, 1/2, 1/3, 1/3, 1/3, 1/2, 1/2, 1/2, 1/2, 0])
Main_Branch_2s.append(y0)
def main_stepfn(y): # Dynamic step size
    if y[-1] < 0.18:
        return 0.02
    else:
        return 0.002
total_steps = 2000
final_l = 2.5

y = y0
for step in range(total_steps):
    determinants = np.array([np.linalg.det(np.delete(Jacobian(y), i, axis=1)) for i in range(12)])
    dy = np.power(-1,np.arange(1,13)) * determinants
    dy = dy/np.linalg.norm(dy) # dy/ds
    ds = main_stepfn(y)
    y_predicted = y + ds * dy # predict new y
    y_optimized = newton_method(H, Jacobian, y_predicted) # fine-grained optimization around prediction
    Main_Branch_2s.append(y_optimized)
    y = y_optimized
    if y[-1] > final_l:
        break
Main_Branch_2s = np.array(Main_Branch_2s)

Side_Branch_2s = []
y0 = np.array([1.00000000e+00, 1.27340731e-21, 1.00000000e+00, 1.91955459e-22,
               1.30995117e-01, 8.69004883e-01, 1.66810219e-22, 2.61938021e-01,
               7.38061979e-01, 1.00000000e+00, 2.64675850e-16, 2.50023108e+00]) # Close to a sequential nash
Side_Branch_2s.append(y0)
def side_stepfn(y): # Dynamic step size
    max_step = 0.01
    mid_step = 0.001
    min_step = 0.0001
    if y[-1] < 0.25:
        return min_step
    elif y[-1] < 1:
        return mid_step
    else:
        return max_step
total_steps = 7000
final_l = 2.5

y = y0
for step in range(total_steps):
    determinants = np.array([np.linalg.det(np.delete(Jacobian(y), i, axis=1)) for i in range(12)])
    dy = np.power(-1,np.arange(1,13)) * determinants
    dy = -dy/np.linalg.norm(dy) # dy/ds
    ds = side_stepfn(y)
    y_predicted = y + ds * dy # predict new y
    y_optimized = newton_method(H, Jacobian, y_predicted) # fine-grained optimization around prediction
    Side_Branch_2s.append(y_optimized)
    y = y_optimized
    if y[-1] > final_l:
        break
Side_Branch_2s = np.array(Side_Branch_2s)

# Save to dataframes
cols = ['pbhh', 'pbdh', 'pbll', 'pbdl', 'pshh', 'pslh', 'pshl', 'psll', 'l']
dfmain_1s = pd.DataFrame(Main_Branch_1s, columns=cols)
dfmain_1s['muh'] = dfmain_1s['pshh'] / (dfmain_1s['pshh'] + dfmain_1s['pshl'])
dfmain_1s['mul'] = (1 - dfmain_1s['pshh']) / (2 - dfmain_1s['pshh'] - dfmain_1s['pshl'])
dfmain_1s.to_excel('../output/QRE/qre_main_1s.xlsx', index=False)

cols = ['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb', 'pshl', 'psll', 'pshh', 'pslh', 'l']
dfmain_2s = pd.DataFrame(Main_Branch_2s, columns=cols)
dfside_2s = pd.DataFrame(Side_Branch_2s, columns=cols)
dfmain_2s['muh'] = dfmain_2s['pshh'] / (dfmain_2s['pshh'] + dfmain_2s['pshl'])
dfmain_2s['mul'] = dfmain_2s['pslh'] / (dfmain_2s['pslh'] + dfmain_2s['psll'])
dfside_2s['muh'] = dfside_2s['pshh'] / (dfside_2s['pshh'] + dfside_2s['pshl'])
dfside_2s['mul'] = dfside_2s['pslh'] / (dfside_2s['pslh'] + dfside_2s['psll'])
dfmain_2s.to_excel('../output/QRE/qre_main_2s.xlsx', index=False)
dfside_2s.to_excel('../output/QRE/qre_side_2s.xlsx', index=False)


<ipython-input-44-cb3734abd2b0>:55: RuntimeWarning: divide by zero encountered in true_divide
  caseE_BH_BL_DB = ((kl * x2 * (1 - np.power(x2, n)) - (1 - x2) * np.power(x2, n))
<ipython-input-44-cb3734abd2b0>:117: RuntimeWarning: divide by zero encountered in true_divide
  caseE_BH_BL_DB = ((kh * x3 * (1 - np.power(x3, n)) - (1 - x3) * np.power(x3, n))
<ipython-input-44-cb3734abd2b0>:152: RuntimeWarning: invalid value encountered in true_divide
  + highmix_BH_BL_DB * ((1 - np.power(1 - highmix_x, highmix_n)) / highmix_x)) / highmix_n)


In [ ]:
#--------------------QRE graphs--------------------#

#--------------------Choice Probs on Precision Figure n = 1--------------------#

dfmain_1s = pd.read_excel('../output/QRE/qre_main_1s.xlsx')

# First cut down dataframe a bit because it's bigger than it needs to be
dfmain1cut = dfmain_1s[['pbhh', 'pbll', 'pshh', 'pshl', 'muh', 'mul', 'l']].iloc[::10, :]

output_file = '../output/QRE/QREprobs_n1.data'
dfmain1cut.to_csv(output_file, sep=',', index=False)
print(f'Data written to {output_file}')

tex_code = r"""
\begin{figure}[htbp]\flushleft

\begin{subfigure}[b]{0.8\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Seller Choice Probabilities}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.21, xtick={0,0.05,0.1,0.15,0.2}, xticklabels={}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(p_H|v_H)$}
\addplot [color=orange, solid] table [x=l, y=pshl, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(p_H|v_L)$}
\end{axis}
\end{tikzpicture}
\end{subfigure}

\begin{subfigure}[b]{0.8\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Buyer Choice Probabilities}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.21, xtick={0,0.05,0.1,0.15,0.2}, xticklabels={}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=pbhh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(\text{buy}|p_H)$}
\addplot [color=orange, solid] table [x=l, y=pbll, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(\text{buy}|p_L)$}
\end{axis}
\end{tikzpicture}
\end{subfigure}

\begin{subfigure}[b]{0.8\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Buyer Beliefs}, xlabel={Precision}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.21, xtick={0,0.05,0.1,0.15,0.2}, xticklabels={0,0.05,0.1,0.15,0.2}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=muh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(v_H|p_H)$}
\addplot [color=orange, solid] table [x=l, y=mul, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addlegendentry{$\mathbb{P}(v_H|p_L)$}
\end{axis}
\end{tikzpicture}
\end{subfigure}

\caption{QRE correspondence for $n=1$}
\label{QREprobs_n1}
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} This figure is based on the actual parameter values used in the experiment. Because firms are symmetric, the average price is simply $x p_H + (1-x) p_L$ where $x$ is the unconditional probability of an individual firm setting the high price.
%\end{minipage}
\end{figure}
"""

with open("../output/QRE/QREprobs_n1.tex", "w") as file:
    file.write(tex_code)
print('Created QREprobs_n1.tex')

#--------------------QRE Seller Strategy Space Figure n = 1--------------------#

eqa = np.load('../output/Theory Section/eqa.npz')
loc_n1_lmix_eq = (eqa['lmix_pi_1s'][6], eqa['lmix_pi_1s'][4])

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}
\begin{axis}[ylabel={$\mathbb{P}(p_H|v_H)$}, xlabel={$\mathbb{P}(p_H|v_L)$}, no markers, ultra thick, xmin=0, xmax=1, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addplot [color=blue, only marks, mark=*, mark size=3] coordinates {<<loc_n1_lmix_eq>>};
\end{axis}
\end{tikzpicture}
\caption{QRE Seller Strategies for $n=1$}
\label{QREsellerstrat_n1}
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} This figure is based on the actual parameter values used in the experiment. Because firms are symmetric, the average price is simply $x p_H + (1-x) p_L$ where $x$ is the unconditional probability of an individual firm setting the high price.
%\end{minipage}
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>": str(loc_n1_lmix_eq)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/QRE/QREsellerstrat_n1.tex", "w") as file:
    file.write(tex_code)
print('Created QREsellerstrat_n1.tex')

#--------------------Choice Probs on Precision Figure n = 2--------------------#

dfmain_2s = pd.read_excel('../output/QRE/qre_main_2s.xlsx')
dfside_2s = pd.read_excel('../output/QRE/qre_side_2s.xlsx')

# The actual dataframes are too big, and include more data than necessary for graphing
# So first cut them down a little
dfmain2cut = dfmain_2s[['pbhh', 'pbll', 'pbhb', 'pblb', 'pshh', 'pshl', 'muh', 'mul', 'l']].iloc[::3, :]
dfside2cut = dfside_2s[['pbhh', 'pbll', 'pbhb', 'pblb', 'pshh', 'pshl', 'muh', 'mul', 'l']].iloc[::10, :]

output_file = '../output/QRE/QREprobsmain_n2.data'
dfmain2cut.to_csv(output_file, sep=',', index=False)
print(f'Data written to {output_file}')

output_file = '../output/QRE/QREprobsside_n2.data'
dfside2cut.to_csv(output_file, sep=',', index=False)
print(f'Data written to {output_file}')

tex_code = r"""
\begin{figure}[htbp]\flushleft
    
\begin{subfigure}[b]{0.7\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Seller Choice Probabilities}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.31, xtick={0,0.05,0.1,0.15,0.2,0.25,0.3}, xticklabels={}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(p_H|v_H)$}
\addplot [color=orange, solid] table [x=l, y=pshl, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(p_H|v_L)$}
\addplot [color=blue, solid] table [x=l, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=orange, solid] table [x=l, y=pshl, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\end{axis}
\end{tikzpicture}
\end{subfigure}

\begin{subfigure}[b]{0.7\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Buyer Choice Probabilities}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.31, xtick={0,0.05,0.1,0.15,0.2,0.25,0.3}, xticklabels={}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=pbhh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(\text{buy}|p_H, \ p_H)$}
\addplot [color=orange, solid] table [x=l, y=pbll, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(\text{buy}|p_L, \ p_L)$}
\addplot [color=blue, dashed] table [x=l, y=pbhb, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(\text{buy} \ p_H|p_H, \ p_L)$}
\addplot [color=orange, dashed] table [x=l, y=pblb, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(\text{buy} \ p_L|p_H, \ p_L)$}
\addplot [color=blue, solid] table [x=l, y=pbhh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=orange, solid] table [x=l, y=pbll, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=blue, dashed] table [x=l, y=pbhb, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=orange, dashed] table [x=l, y=pblb, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\end{axis}
\end{tikzpicture}
\end{subfigure}

\begin{subfigure}[b]{0.7\textwidth}
\begin{tikzpicture}
\begin{axis}[ylabel={Buyer Beliefs}, xlabel={Precision}, width=\textwidth, height=0.3\textheight, no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, ultra thick, xmin=-0.01, xmax=0.31, xtick={0,0.05,0.1,0.15,0.2,0.25,0.3}, xticklabels={0,0.05,0.1,0.15,0.2,0.25,0.3}, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=l, y=muh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(v_H|p_H)$}
\addplot [color=orange, solid] table [x=l, y=mul, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addlegendentry{$\mathbb{P}(v_H|p_L)$}
\addplot [color=blue, solid] table [x=l, y=muh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=orange, solid] table [x=l, y=mul, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\end{axis}
\end{tikzpicture}
\end{subfigure}

\caption{QRE correspondence for $n=2$}
\label{QREprobs_n2}
%\begin{minipage}{0.7\textwidth}
%\footnotesize
%\textit{Notes:} This figure is based on the actual parameter values used in the experiment. Because firms are symmetric, the average price is simply $x p_H + (1-x) p_L$ where $x$ is the unconditional probability of an individual firm setting the high price.
%\end{minipage}
\end{figure}
"""

with open("../output/QRE/QREprobs_n2.tex", "w") as file:
    file.write(tex_code)
print('Created QREprobs_n2.tex')

#--------------------QRE Seller Strategy Space Figure n = 2--------------------#

loc_n2_lmix_eq = (eqa['lmix_pi_2s'][9], eqa['lmix_pi_2s'][7])
loc_n2_hmix_eq = (eqa['hmix_pi_2s'][9], eqa['hmix_pi_2s'][7])
loc_n2_pool_eq = (eqa['pool_pi_2s'][9], eqa['pool_pi_2s'][7])

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}
\begin{axis}[ylabel={$\mathbb{P}(p_H|v_H)$}, xlabel={$\mathbb{P}(p_H|v_L)$}, no markers, ultra thick, xmin=0, xmax=1, ymin=0, ymax=1]
\addplot [color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsmain_n2.data};
\addplot [color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobsside_n2.data};
\addplot [color=blue, only marks, mark=*, mark size=3] coordinates {<<loc_n2_lmix_eq>>};
\addplot [color=blue, only marks, mark=*, mark size=3] coordinates {<<loc_n2_hmix_eq>>};
\addplot [color=blue, only marks, mark=*, mark size=3] coordinates {<<loc_n2_pool_eq>>};
\end{axis}
\end{tikzpicture}
\caption{QRE Seller Strategies for $n=2$}
\label{QREsellerstrat_n2}
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} This figure is based on the actual parameter values used in the experiment. Because firms are symmetric, the average price is simply $x p_H + (1-x) p_L$ where $x$ is the unconditional probability of an individual firm setting the high price.
%\end{minipage}
\end{figure}
"""

replacements = {"<<loc_n2_lmix_eq>>": str(loc_n2_lmix_eq),
                "<<loc_n2_hmix_eq>>": str(loc_n2_hmix_eq),
                "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/QRE/QREsellerstrat_n2.tex", "w") as file:
    file.write(tex_code)
print('Created QREsellerstrat_n2.tex')


In [ ]:
#--------------------Finite Mixture Model--------------------#

print('Fitting finite mixture model...')

#--------------------Interpolate between QRE array values--------------------#

cln_data_path = '../output/Data Background/'
bysess_all_1s = pd.read_excel(cln_data_path + 'bysess_all_1s_first.xlsx')
bysess_all_2s = pd.read_excel(cln_data_path + 'bysess_all_2s_first.xlsx')

data_1s = bysess_all_1s[['pbhh', 'pbdh', 'pbll', 'pbdl', 
                        'pshh', 'pslh', 'pshl', 'psll']].values
data_2s = bysess_all_2s[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb', 
                        'pshh', 'pslh', 'pshl', 'psll']].values

dfmain_1s = pd.read_excel('../output/QRE/qre_main_1s.xlsx')
dfmain_2s = pd.read_excel('../output/QRE/qre_main_2s.xlsx')
dfside_2s = pd.read_excel('../output/QRE/qre_side_2s.xlsx')

dfmain_1s = dfmain_1s[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pshh', 'pslh', 'pshl', 'psll', 'l']]
dfmain_2s = dfmain_2s[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb', 'pshh', 'pslh', 'pshl', 'psll', 'l']]
dfside_2s = dfside_2s[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb', 'pshh', 'pslh', 'pshl', 'psll', 'l']]

min_index = dfside_2s['l'].idxmin()
dfupside_2s = dfside_2s.iloc[:min_index + 1]
dfdownside_2s = dfside_2s.iloc[min_index:]
threshold_l = dfside_2s['l'].min()

def interpolate_df(df, l):
    choiceprobs = []
    lower_row = df[df['l'] <= l].iloc[-1]
    upper_row = df[df['l'] > l].iloc[0]
    l1 = lower_row['l']
    l2 = upper_row['l']
    for col in [x for x in df.columns if x != 'l']:
        y1 = lower_row[col]
        y2 = upper_row[col]
        y_value = y1 + (l - l1) * (y2 - y1) / (l2 - l1)
        choiceprobs.append(y_value)
    return np.array(choiceprobs)

def QREchoiceprobs_1s(l):
    return np.array([interpolate_df(dfmain_1s, l)])

def QREchoiceprobs_2s(l):
    eq_main_2s = interpolate_df(dfmain_2s, l)
    if l >= threshold_l:
        eq_upside_2s   = interpolate_df(dfupside_2s, l)
        eq_downside_2s = interpolate_df(dfdownside_2s, l)
        return np.array([eq_main_2s, eq_upside_2s, eq_downside_2s])
    else:
        return np.array([eq_main_2s])

#--------------------Minimize Negative Log Likelihood--------------------#

from scipy.optimize import minimize

def NLL_1s(l):
    l = l[0]
    QRE_choiceprobs_1s = np.expand_dims(QREchoiceprobs_1s(l), axis=0)
    logsum_1s = (np.expand_dims(data_1s, axis=1) * np.log(QRE_choiceprobs_1s)).sum(axis=2)
    lnlikelihood_1s = logsum_1s.sum(axis=0)
    return -lnlikelihood_1s[0]

def NLL_2s_(l):
    l = l[0]
    QRE_choiceprobs_2s = np.expand_dims(QREchoiceprobs_2s(l), axis=0)
    logsum_2s = (np.expand_dims(data_2s, axis=1) * np.log(QRE_choiceprobs_2s)).sum(axis=2)
    lnlikelihood_2s = logsum_2s.sum(axis=0)
    return -lnlikelihood_2s[0]

def NLL_2s(params):
    l = params[0]
    q = np.expand_dims(params[1:], axis=0)
    
    QRE_choiceprobs_2s = np.expand_dims(QREchoiceprobs_2s(l), axis=0)
    
    logsum_2s = (np.expand_dims(data_2s, axis=1) * np.log(QRE_choiceprobs_2s)).sum(axis=2)
    lnlikelihood_ij_2s = logsum_2s + np.log(q) * (l >= threshold_l)
    
    lnlikelihood_ij_2s = lnlikelihood_ij_2s + 200 #go up 200 orders of magnitude so we don't max out float precision
    likelihood_i_2s = (np.exp(lnlikelihood_ij_2s)).sum(axis=1)
    lnlikelihood_i_2s = np.log(likelihood_i_2s) - 200 #and now go back down
    lnlikelihood_2s = lnlikelihood_i_2s.sum(axis=0)
    return -lnlikelihood_2s

result = minimize(NLL_1s, 0.2, bounds=[(0.0001,0.6)])
pi_1s = QREchoiceprobs_1s(result.x[0])[0]
inf_1s = 0.5 * (1 - pi_1s[6] + pi_1s[4])

result = minimize(NLL_2s_, 0.2, bounds=[(0.0001,0.6)])
pi_2s = QREchoiceprobs_2s(result.x[0])[0]
inf_2s = 0.5 * (1 - pi_2s[9] + pi_2s[7])

#--------------------QRE MLE graph--------------------#

loc_n1_lmix_eq = (eqa['lmix_pi_1s'][6], eqa['lmix_pi_1s'][4])
loc_n2_lmix_eq = (eqa['lmix_pi_2s'][9], eqa['lmix_pi_2s'][7])
loc_n2_hmix_eq = (eqa['hmix_pi_2s'][9], eqa['hmix_pi_2s'][7])
loc_n2_pool_eq = (eqa['pool_pi_2s'][9], eqa['pool_pi_2s'][7])
bestfitQRE_n1 = (pi_1s[6], pi_1s[4])
bestfitQRE_n2 = (pi_2s[9], pi_2s[7])

tex_code = r"""
\begin{figure}[htbp]\flushleft

\begin{subfigure}[b]{0.45\textwidth}
\begin{tikzpicture}[scale=0.85]
\begin{axis}[ylabel={$\mathbb{P}(p_H|v_H)$}, xlabel={$\mathbb{P}(p_H|v_L)$}, ultra thick, xmin=0, xmax=1, ymin=0, ymax=1]
\addplot[no markers, color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump] {../output/QRE/QREprobs_n1.data};
\addplot[no markers, color=blue, only marks, mark=*, mark size=2] coordinates {<<loc_n1_lmix_eq>>};
\addplot[only marks, mark=*, mark options={color=black, fill=black}] table[x=pshl_1s, y=pshh_1s, col sep=comma] {../output/Results Section/sellerstrat_bysession_first.data}; 
\node[star,star points=5,star point ratio=2.5,scale=0.7,draw] at <<bestfitQRE_n1>> {} ;
\end{axis}
\end{tikzpicture}
\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.45\textwidth}
\begin{tikzpicture}[scale=0.85]
\begin{axis}[ylabel={}, xlabel={$\mathbb{P}(p_H|v_L)$}, ytick={}, yticklabels={}, ultra thick, xmin=0, xmax=1, ymin=0, ymax=1, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align=left]
\addplot[no markers, color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump, forget plot] {../output/QRE/QREprobsmain_n2.data};
\addplot[no markers, color=blue, solid] table [x=pshl, y=pshh, col sep=comma, unbounded coords=jump, forget plot] {../output/QRE/QREprobsside_n2.data};
\addplot[color=blue, only marks, mark=*, mark size=2, forget plot] coordinates {<<loc_n2_lmix_eq>>};
\addplot[color=blue, only marks, mark=*, mark size=2, forget plot] coordinates {<<loc_n2_hmix_eq>>};
\addplot[color=blue, only marks, mark=*, mark size=2, forget plot] coordinates {<<loc_n2_pool_eq>>};
\addplot[only marks, mark=*, mark options={color=black, fill=black}] table[x=pshl_2s, y=pshh_2s, col sep=comma, forget plot] {../output/Results Section/sellerstrat_bysession_first.data}; 
\addlegendimage{only marks, mark=*, black}
\addlegendentry{Data}
\addlegendimage{mark=none, legend image code/.code={%
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw] at (0.3cm,0) {};
}}
\addlegendentry{Best-fit QRE}
\node[star,star points=5,star point ratio=2.5,scale=0.7,draw] at <<bestfitQRE_n2>> {} ;
\end{axis}
\end{tikzpicture}
\caption{$n = 2$}
\end{subfigure}

\caption{QRE Seller Strategies}
\label{qresellerstratmle}
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} Notice this notable note denotes noteworthy notices.
%\end{minipage}
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>": str(loc_n1_lmix_eq),
                "<<loc_n2_lmix_eq>>": str(loc_n2_lmix_eq),
                "<<loc_n2_hmix_eq>>": str(loc_n2_hmix_eq),
                "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq),
                "<<bestfitQRE_n1>>": str(bestfitQRE_n1),
                "<<bestfitQRE_n2>>": str(bestfitQRE_n2)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/QRE/QREsellerstratmle.tex", "w") as file:
    file.write(tex_code)
print('Created QREsellerstratmle.tex')

print('Informativeness in best-fit QRE:')
print('n = 1: ', str(inf_1s))
print('n = 2: ', str(inf_2s))
print('\n')
